This notebook contains Python code for reproducing the results in our paper on using a large language model to assess the quality of automatically generated questions:

Dittel, J. S., Van Campenhout, R., Clark, M. W., & Johnson, B. G. (2024). Exploring Large Language Models for Evaluating Automatically Generated Questions. 25th International Conference on Artificial Intelligence in Education (AIED 2024), Workshop on Automated Evaluation of Learning and Assessment Content, Recife, Brazil, July 2024. https://drive.google.com/file/d/1vO21K60lDf18izQdr79CpJxOvfXvHQBM/view

Results are presented in the order they occur, organized by the paper's sections. For each result, an excerpt from the paper is given followed by code to compute the result from the data set provided. Example:

> The mean number of students answering each question was 118.3 and the overall question mean score on the first attempt was 41.2%.

```print( f'{question_data.students.mean():.1f} {question_data.mean_score.mean():.1%}' )```

Please refer to the paper for additional context.

In [1]:
import os

import openai
import pandas as pd
import sklearn.metrics as skm
from jellyfish import levenshtein_distance
from statsmodels.stats.proportion import proportions_ztest

## 2. Methods

### Descriptive statistics

In [2]:
question_data = pd.read_csv( 'question_data.csv', index_col='question_id' )
question_data.shape

(54, 13)

In [3]:
question_data.head()

,stem,answer,students,mean_score,thumbs_down,reject_thumbs_down,reject_mean_score,reject,answer_stem,answer_stem_correct,paragraph,answer_paragraph,answer_paragraph_correct
question_id,,,,,,,,,,,,,
27c7e28f0678ec6a1aacb0e4860b480961f0e72d65e1a5bdfb368b4b1a4d9a6c,Chemists usually perform experiments under nor...,constant,88,0.329545,3,True,True,True,constant,True,The heat given off when you operate a Bunsen b...,constant,True
e5d7596ad237b4a80be9b62b3855d3b49d811de08858c0ca15bc88b71a4d9a6c,Chemists ordinarily use a property known as en...,thermodynamics,89,0.078652,3,True,True,True,energy,False,Chemists ordinarily use a property known as en...,energy,False
ef4e7e437722682521d9e5498a3559211dfa5ad8e114bb9ef2bfc2f31a4d9a6c,The large scale separation of ________ 235UF6 ...,gaseous,91,0.087912,3,True,True,True,isotope,False,The large scale separation of ________ 235UF6 ...,isotope,False
e005a4275d74dd562fc9d7222cafaeb785952fe392a598a233831dc71a4d9a6c,The following guidelines are used to assign __...,oxidation,95,0.305263,1,False,True,True,oxidation,True,The product of this reaction is a covalent com...,oxidation,True
1e9f9264c12535f9155c06000e9b3324e063d6cb07e24a04eddba3121a4d9a6c,"For a(n) ________ molecule, the atomic orbital...",diatomic,96,0.333333,1,False,True,True,diatomic,True,The relative energy levels of atomic and molec...,diatomic,True


In [4]:
events = pd.read_parquet( 'events.parquet' )
events.shape

(12373, 7)

In [5]:
events.head()

,timestamp,user_id,event_type,question_id,correct,first_attempt,extra
0,2023-09-05 20:46:16,8CUZE5QBGJZGPCC4V8TJ,evaluate_response,c9587d4976f87afff9c3c65df9ee73667149bbd57913ce...,False,True,"{'assessment_id': None, 'assessment_type': 'fo..."
1,2023-09-05 20:46:21,8CUZE5QBGJZGPCC4V8TJ,reveal_answer,c9587d4976f87afff9c3c65df9ee73667149bbd57913ce...,None,None,"{'assessment_id': None, 'assessment_type': 'fo..."
2,2023-09-05 21:00:34,5RFDR3ZHBG7XC4QM8MV3,evaluate_response,1871daad50bfdba1970295c669f140b73cbbfe8766bb00...,True,True,"{'assessment_id': None, 'assessment_type': 'fo..."
3,2023-09-05 21:17:43,5Q58F5PDXXFHJJHNJ5FW,evaluate_response,1871daad50bfdba1970295c669f140b73cbbfe8766bb00...,False,True,"{'assessment_id': None, 'assessment_type': 'fo..."
4,2023-09-05 21:17:46,5Q58F5PDXXFHJJHNJ5FW,reveal_answer,1871daad50bfdba1970295c669f140b73cbbfe8766bb00...,None,None,"{'assessment_id': None, 'assessment_type': 'fo..."


> The mean number of students answering each question was 118.3 and the overall question mean score on the first attempt was 41.2%.

In [6]:
print( f'{question_data.students.mean():.1f} {question_data.mean_score.mean():.1%}' )

118.3 41.2%


> A total of 247 students answered questions in the data set.

In [7]:
print( events.user_id.nunique() )

247


> The mean number of questions answered per student was 25.9, with 71 students answering all 54 questions in the data set.

In [8]:
questions_per_student = events.groupby( 'user_id' ).question_id.nunique()
print( f"{questions_per_student.mean():.1f} {( questions_per_student == len( question_data ) ).sum()}" )

25.9 71


> Of these 54 questions, 27 (50%) would be flagged for rejection using one or both of the CIS criteria (21 by mean score and 13 by student ratings).

In [9]:
print( question_data.reject.sum(), question_data.reject_mean_score.sum(), question_data.reject_thumbs_down.sum() )

27 21 13


### Example of answering a question using the LLM

In [10]:
openai.api_key = os.getenv( 'OPENAI_API_KEY' )

In [11]:
def get_message( content, role='user' ):
    assert role in [ 'system', 'user', 'assistant' ], f'Bad role: "{role}"'
    return { "role": role, "content": content } 

def get_completion( role, prompt, temperature ):
    completion = openai.ChatCompletion.create(
      model='gpt-4',
      temperature=temperature,
      messages=[
        get_message( role, role='system' ),
        get_message( prompt, role='user' ),
      ]
    )
    return completion

In [12]:
def answer_fitb_question( stem, temperature=0 ):
    role = 'You are a college student with a 4.0 GPA.'
    prompt = f"""Here is a fill-in-the-blank question from your textbook. Please answer with the word that best fits in the blank. Answer only with a single word.

Question: {stem}
Answer:"""
    completion = get_completion( role, prompt, temperature )
    answer = completion.choices[ 0 ].message.content
    return answer

> The uncertainty principle can be shown to be a consequence of wave-particle duality, which lies at the heart of what distinguishes modern ______ theory from classical mechanics.

In [13]:
stem = 'The uncertainty principle can be shown to be a consequence of wave-particle duality, which lies at the heart of what distinguishes modern ______ theory from classical mechanics.'
answer = answer_fitb_question( stem )
print( f'Question: {stem}\nAnswer: {answer}' )

Question: The uncertainty principle can be shown to be a consequence of wave-particle duality, which lies at the heart of what distinguishes modern ______ theory from classical mechanics.
Answer: quantum


## 3. Results and Discussion

### Success rate of LLM answering questions

> When the LLM was given each question to answer without additional context, 43 of 54 (79.6%) were answered correctly.

In [14]:
print( question_data.answer_stem_correct.sum(), f'{question_data.answer_stem_correct.mean():.1%}' )

43 79.6%


Answer questions with LLM using the question stem.

In [15]:
def assess_answer( answer, correct_answer ):
    # Answer check is case-insensitive and ignores simple singular-plural differences
    answer = answer.lower().strip() # '\n' appended sometimes by LLM
    correct_answer = correct_answer.lower()
    correct = answer == correct_answer
    if not correct:
        # Check for simple singular-plural difference
        d = levenshtein_distance( answer, correct_answer )
        correct = d <= 1
    if not correct:
        # Check -y/-ies difference
        if answer.endswith( 'y' ) and correct_answer.endswith( 'ies' ):
            correct = answer[ :-1 ] == correct_answer[ :-3 ]
        elif answer.endswith( 'ies' ) and correct_answer.endswith( 'y' ):
            correct = answer[ :-3 ] == correct_answer[ :-1 ]
    return correct

In [16]:
for stem, correct_answer, answer_stem_correct in zip( question_data.stem, question_data.answer, question_data.answer_stem_correct ):
    answer = answer_fitb_question( stem )
    correct = assess_answer( answer, correct_answer )
    print( stem )
    print( 'Correct answer:', correct_answer )
    print( 'LLM answer:    ', answer, 'CORRECT' if correct else 'INCORRECT' )
    print()

Chemists usually perform experiments under normal atmospheric conditions, at ________ external pressure with q = ΔH, which makes enthalpy the most convenient choice for determining heat changes for chemical reactions.
Correct answer: constant
LLM answer:     constant CORRECT

Chemists ordinarily use a property known as enthalpy (H) to describe the ________ of chemical and physical processes.
Correct answer: thermodynamics
LLM answer:     energy INCORRECT

The large scale separation of ________ 235UF6 from 238UF6 was first done during the World War II, at the atomic energy installation in Oak Ridge, Tennessee, as part of the Manhattan Project (the development of the first atomic bomb).
Correct answer: gaseous
LLM answer:     Isotope INCORRECT

The following guidelines are used to assign ________ numbers to each element in a molecule or ion.
Correct answer: oxidation
LLM answer:     oxidation CORRECT

For a(n) ________ molecule, the atomic orbitals of one atom are shown on the left, and 

> This is in stark contrast with the first attempts by students, which were only 41.2% correct.

In [17]:
print( f'{question_data.mean_score.mean():.1%}' )

41.2%


> A _z_ test of two proportions shows this difference is statistically significant (_p_ << .001).

In [18]:
z, p = proportions_ztest( count=[ question_data.answer_stem_correct.sum(), question_data.mean_score.sum() ],
                          nobs=[ len( question_data ) ] * 2 )
print( f'z = {z:.2f}, p = {p:.2e}' )

z = 4.08, p = 4.48e-05


### Precision and recall

> Taking an incorrect answer as predicting rejection by the CIS had precision 72.7% and recall
29.6%.

In [19]:
precision = skm.precision_score( question_data.reject, ~question_data.answer_stem_correct )
recall = skm.recall_score( question_data.reject, ~question_data.answer_stem_correct )
print( f'precision = {precision:.1%}, recall = {recall:.1%}' )

precision = 72.7%, recall = 29.6%


### Impact of providing additional context

> Providing the textbook paragraph in which the sentence occurred as context is expected to
increase the proportion of questions correctly answered. This was observed, with 46 of 54 (85.2%) correct, compared to 79.6% correct without this contextual information.

In [20]:
print( question_data.answer_paragraph_correct.sum(), f'{question_data.answer_paragraph_correct.mean():.1%}' )

46 85.2%


Answer questions with LLM using the question sentence's paragraph as context.

In [21]:
for paragraph, correct_answer, answer_paragraph_correct in zip( question_data.paragraph, question_data.answer, question_data.answer_paragraph_correct ):
    answer = answer_fitb_question( paragraph )
    correct = assess_answer( answer, correct_answer )
    print( paragraph )
    print( 'Correct answer:', correct_answer )
    print( 'LLM answer:    ', answer, 'CORRECT' if correct else 'INCORRECT' )
    print()

The heat given off when you operate a Bunsen burner is equal to the enthalpy change of the methane combustion reaction that takes place, since it occurs at the essentially constant pressure of the atmosphere. On the other hand, the heat produced by a reaction measured in a bomb calorimeter (Figure 9.17) is not equal to ΔH because the closed, constant-volume metal container prevents the pressure from remaining constant (it may increase or decrease if the reaction yields increased or decreased amounts of gaseous species). Chemists usually perform experiments under normal atmospheric conditions, at ________ external pressure with q = ΔH, which makes enthalpy the most convenient choice for determining heat changes for chemical reactions.
Correct answer: constant
LLM answer:     constant CORRECT

Chemists ordinarily use a property known as enthalpy (H) to describe the ________ of chemical and physical processes. Enthalpy is defined as the sum of a system's internal energy (U) and the mathem

> More correctly answered questions means fewer questions predicted as rejected, and so recall should decrease and precision increase. While this was the case, with precision 75.0% and recall 22.2%, interestingly, the difference made by this additional information was small.

In [22]:
precision = skm.precision_score( question_data.reject, ~question_data.answer_paragraph_correct )
recall = skm.recall_score( question_data.reject, ~question_data.answer_paragraph_correct )
print( f'precision = {precision:.1%}, recall = {recall:.1%}' )

precision = 75.0%, recall = 22.2%


### Specific examples discussed in the paper

> An example of a question correctly predicted for rejection by the CIS is
>
> The order of a(n) ______ bond is a guide to its strength; a bond between two given atoms becomes stronger as the bond order increases.
> 
> The correct answer is “covalent” and the LLM’s answer was “chemical”. While both are reasonable words for completing the sentence in isolation, “covalent” is more specific to the textbook context on the topic of nolecular orbitals for diatomic molecules, which concerns covalent bonding.

In [23]:
question_data[ question_data.stem.str.startswith( 'The order of' ) ]

,stem,answer,students,mean_score,thumbs_down,reject_thumbs_down,reject_mean_score,reject,answer_stem,answer_stem_correct,paragraph,answer_paragraph,answer_paragraph_correct
question_id,,,,,,,,,,,,,
2951446f9ba716169c44ad13d2ac573e74ab3a1b6ec1caac36c2c9bc1a4d9a6c,The order of a(n) ________ bond is a guide to ...,covalent,98,0.367347,2,True,False,True,Chemical,False,The order of a(n) ________ bond is a guide to ...,Molecular,False


> Another question correctly predicted for rejection is
> 
> One particularly characteristic ______ of waves results when two or more waves come into contact: They interfere with each other.
> 
> The correct answer word (i.e., appearing in the textbook sentence) is “phenomenon” while the LLM answered “property”. Here, these words are synonymous, completing the sentence equally well (even considering context), but “property” would be counted as incorrect. This illustrates how the LLM-based answer criterion can be useful, by identifying when an equally good answer word as the one used by the textbook author exists.

In [24]:
question_data[ question_data.stem.str.startswith( 'One particularly characteristic' ) ]

,stem,answer,students,mean_score,thumbs_down,reject_thumbs_down,reject_mean_score,reject,answer_stem,answer_stem_correct,paragraph,answer_paragraph,answer_paragraph_correct
question_id,,,,,,,,,,,,,
e9816d2e6d1319340024bb1ac20661cf2406b3b5bd6585da5f4a554b1a4d9a6c,One particularly characteristic ________ of wa...,phenomenon,128,0.359375,3,True,False,True,property,False,One particularly characteristic ________ of wa...,phenomenon,True


> An example question that the LLM answered correctly but was given thumbs down by multiple students illustrates this point.
>
> Because a hydrogen ______ molecule contains two oxygen atoms, as opposed to the water molecule, which has only one, the two substances exhibit very different properties.
>
> The LLM correctly answered “peroxide”, which is highly predictable in this context, and thus the question was not predicted for rejection. However, students viewed the question as not helpful because this sentence was serving as an example to illustrate a central concept (the chemical mole concept), and not as an important fact that needed to be retained. This reason was not related to the answer word’s predictability.

In [25]:
question_data[ question_data.stem.str.startswith( 'Because a hydrogen' ) ]

,stem,answer,students,mean_score,thumbs_down,reject_thumbs_down,reject_mean_score,reject,answer_stem,answer_stem_correct,paragraph,answer_paragraph,answer_paragraph_correct
question_id,,,,,,,,,,,,,
ad6f37c4ae3f508474f1260afa85ec1a416615c65992d92962e855b81a4d9a6c,Because a hydrogen ________ molecule contains ...,peroxide,119,0.184874,2,True,True,True,peroxide,True,The identity of a substance is defined not onl...,peroxide,True
